In [158]:
startBook = "<book"
books = []
with open("../data/BlurbGenreCollection_EN_train.txt") as openFile:
    for line in openFile:
        if line[:len(startBook)] == startBook:
            books.append(line)
        else:
            books[-1] += line

print(books[0])


<book date="2018-08-18" xml:lang="en"> 
<title>The New York Times Daily Crossword Puzzles: Thursday, Volume 1</title>
<body>Monday’s Crosswords Do with EaseTuesday’s Crosswords Not a BreezeWednesday’s Crosswords Harder StillThursday’s Crosswords Take Real SkillFriday’s Crosswords — You’ve Come This Far…Saturday’s Crosswords — You’re a Star!For millions of people, the New York Times crossword puzzles are as essential to each day as the first cup of coffee in the morning. Now, for the first time ever, these premier puzzles are available in six clever installments. With each day of the week, the puzzles increase gradually in skill level; Monday’s the easiest, but Saturday’s sure to challenge! Push your mental muscles a little harder each day with America’s favorite sophisticated — and fun — pastime: the New York Times crossword puzzles!The legendary Eugene T. Maleska was crossword editor of The New York Times from 1977 to 1993.</body>
<copyright>(c) Penguin Random House</copyright>
<metad

In [157]:
synopses = []
genres = []

startBody = "<body>"
endBody = "</body>"

startTopic = "<topics>"
endTopic = "</topics>"

for book in books:
    start = book.index(startBody) + len(startBody)
    end = book.index(endBody)
    synopses.append(book[start:end])

    start = book.index(startTopic) + len(startTopic)
    end = book.index(endTopic)
    genres.append(book[start:end])


print(synopses[0])
print(genres[0])
print(synopses[1])
print(genres[1])

Monday’s Crosswords Do with EaseTuesday’s Crosswords Not a BreezeWednesday’s Crosswords Harder StillThursday’s Crosswords Take Real SkillFriday’s Crosswords — You’ve Come This Far…Saturday’s Crosswords — You’re a Star!For millions of people, the New York Times crossword puzzles are as essential to each day as the first cup of coffee in the morning. Now, for the first time ever, these premier puzzles are available in six clever installments. With each day of the week, the puzzles increase gradually in skill level; Monday’s the easiest, but Saturday’s sure to challenge! Push your mental muscles a little harder each day with America’s favorite sophisticated — and fun — pastime: the New York Times crossword puzzles!The legendary Eugene T. Maleska was crossword editor of The New York Times from 1977 to 1993.

<d0>Nonfiction</d0><d1>Games</d1>

Two of literary comics modern masters present a pair of magical and disturbing stories of strange creatures who are not quite what they seem! In The 